In [ ]:
import calendar
import itertools
import sqlite3
import time

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from nba_api import *
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams

pd.options.mode.chained_assignment = None  # default='warn'


MONTHS = list(calendar.month_name)
MONTHS.remove("")
MONTHS = list(map(lambda x: x.lower(), MONTHS))

GAMEDAYS = 82  # minimum number of games a team plays in a season
GAMES_PER_DAY = 15  # number of games per gameday

In [ ]:
con = sqlite3.connect("nba_api.sqlite")

In [ ]:
# create teams table
pd.DataFrame(teams.get_teams()).set_index("id").to_sql("teams", con, if_exists="replace")

In [ ]:
# get attendance stats
seasons = range(2005, 2019)
for season, month in list(itertools.product(seasons, MONTHS))[9:]:
    print(season, month, end="\r")
    r = requests.get(f"https://www.basketball-reference.com/leagues/NBA_{season}_games-{month}.html")
    time.sleep(3)
    soup = BeautifulSoup(r.content, "html.parser")
    schedule = soup.find(id="schedule")
    try:
        tr = schedule.find_all("tr")
    except:
        continue
    data = [[th.getText() for th in tr[i].findAll("th")] for i in range(len(tr))]
    dates = np.array(data[1:])
    columns = ["date", "start_time_ET", "visitor", "visitor_points", "home", "home_points", "box_score", "overtime", "attendance", "arena", "notes"]
    dates_df = pd.DataFrame(data=dates, columns=[columns[0]])

    data = [[td.getText() for td in tr[i].findAll("td")] for i in range(len(tr))]
    data_df = pd.DataFrame(data=data[1:], columns=columns[1:])
    data_df.drop(["box_score", "notes"], axis=1, inplace=True)
    df = pd.concat([dates_df, data_df], axis=1)

    df["time"] = df["date"].map(str) + " " + df["start_time_ET"].map(str)

    # add playoffs column
    try:
        playoffs = df.loc[df["date"] == "Playoffs"].index.values[0]
        df = df.drop(df.index[playoffs]).reset_index(drop=True)
        df["playoffs"] = np.where(df.index.values < playoffs, False, True)
    except:
        df["playoffs"] = False

    df.drop(["date", "start_time_ET"], axis=1, inplace=True)

    df.to_sql(f"{season}_{month}", con, if_exists="replace", index=False)

In [ ]:
%%time
all_data_df = pd.DataFrame()
for season, month in itertools.product(seasons, MONTHS):
    try:
        df = pd.read_sql_query(f'select * from "{season}_{month}"', con)
    except pd.io.sql.DatabaseError:
        continue
    df["time"] = pd.to_datetime(df["time"].values, infer_datetime_format=True)
    df["season"] = df.apply(lambda row: row["time"].year if row["time"].month < 8 else row["time"].year + 1, axis=1)

    all_data_df = all_data_df.append(df, ignore_index=True)
    all_data_df = all_data_df.sort_values("time")

all_data_df.to_sql("raw_game_data", con, index=False, if_exists="replace")

In [ ]:
%%time
df = pd.read_sql_query(f"select * from raw_game_data", con)

# adapt types
df["visitor_points"] = df.visitor_points.astype(int)
df["home_points"] = df.home_points.astype(int)
df["attendance"] = df.attendance.astype(int)
df["time"] = pd.to_datetime(df["time"].values, infer_datetime_format=True)

df["last_five"] = 0
df["season_high_attendance"] = 0

for year in sorted(all_data_df["time"].dt.year.unique())[1:]:
    for team in df.home.unique():
        # define useful filters
        current_season = df["season"] == year
        previous_season = df["season"] == year - 1
        current_team_home = df["home"] == team
        current_team = (df["home"] == team) | (df["visitor"] == team)

        # dummy variable for wins by current_team
        df["win"] = 0
        df.loc[
            (current_season | previous_season)
            & current_team
            & (
                ((df["home"] == team) & (df["visitor_points"] < df["home_points"]))
                | ((df["visitor"] == team) & (df["visitor_points"] > df["home_points"]))
            ),
            "win",
        ] = 1

        # compute win percentage during the season (e.g. on matchday 12, thi is the win percentage over first 12 games)
        df.loc[current_season & current_team_home, "curr_season_win_pct"] = (
            df.loc[current_season & current_team, "win"].expanding().mean()[current_team_home]
        )

        # compute number of wins in last 5/10/15 games
        for i in range(5, 16, 5):
            df.loc[current_season & current_team_home, f"last_{i}"] = (
                df.loc[current_season & current_team, "win"].rolling(i, min_periods=i).sum()[current_team_home]
            )

        try:
            df.loc[current_team_home & current_season, "season_high_attendance"] = df.loc[current_team_home & current_season, "attendance"].max()
        except:
            continue

        df.loc[current_season & current_team_home, "previous_season_win_pct"] = df.loc[previous_season & current_team, "win"].mean()
        df.loc[current_season & current_team_home, "previous_regular_season_win_pct"] = df.loc[
            previous_season & current_team & df["playoffs"] == 1, "win"
        ].mean()
df = df.drop(["win"], axis=1)

df.to_sql("game_data", con, index=False, if_exists="replace")

In [ ]:
championships_df = pd.DataFrame([(year, teams.find_teams_by_championship_year(year)) for year in range(1949, 2023)], columns=["year", "team"])
championships_df["total_championships"] = 1
championships_df["total_championships"] = championships_df.groupby("team")["total_championships"].cumsum()
championships_df.to_sql("championship_winners", con, if_exists="replace", index=False)